In [1]:
import os
import re
import glob
import random
import warnings

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score, classification_report

from tqdm.auto import tqdm

warnings.filterwarnings(action='ignore')

In [2]:
df_forehead = pd.read_csv("new_df_forehead.csv", index_col = 0)

# NaN 값을 0이나 다른 값으로 채운 후 변환
df_forehead['forehead_wrinkle'] = df_forehead['forehead_wrinkle'].fillna(0).astype(int)

# NaN을 유지하면서 변환 (nullable Int64 type)
df_forehead['forehead_wrinkle'] = df_forehead['forehead_wrinkle'].astype('Int64')

# NaN 값을 0이나 다른 값으로 채운 후 변환
df_forehead['adj_forehead_wrinkle_2'] = df_forehead['adj_forehead_wrinkle_2'].fillna(0).astype(int)

# NaN을 유지하면서 변환 (nullable Int64 type)
df_forehead['adj_forehead_wrinkle_2'] = df_forehead['adj_forehead_wrinkle_2'].astype('Int64')


# df_forehead['image_name'] = df_forehead['image_name'].apply(lambda x: "_".join(x.split('_')[0:3]) + '.jpg')

df_forehead

,id,gender,age,angle,width,height,facepart,bbox_x_min,bbox_y_min,bbox_x_max,bbox_y_max,forehead_wrinkle,partname,image_name,label_name,adj_forehead_wrinkle,adj_forehead_wrinkle_2
1,1,F,55,F,2136,3216,1,469.0,661.0,1638.0,1197.0,3,이마,0001_01_F.jpg,0001_01_F_01.json,1.0,1
10,1,F,55,L15,2136,3216,1,369.0,595.0,1540.0,1176.0,3,이마,0001_01_L15.jpg,0001_01_L15_01.json,1.0,1
19,1,F,55,L30,2136,3216,1,404.0,561.0,1427.0,1159.0,3,이마,0001_01_L30.jpg,0001_01_L30_01.json,1.0,1
28,1,F,55,R15,2136,3216,1,494.0,573.0,1772.0,1131.0,3,이마,0001_01_R15.jpg,0001_01_R15_01.json,1.0,1
37,1,F,55,R30,2136,3216,1,744.0,577.0,1755.0,1175.0,3,이마,0001_01_R30.jpg,0001_01_R30_01.json,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43381,1100,F,28,F,2136,3216,1,504.0,879.0,1629.0,1248.0,1,이마,1100_01_F.jpg,1100_01_F_01.json,0.0,0
43390,1100,F,28,L15,2136,3216,1,339.0,847.0,1400.0,1201.0,1,이마,1100_01_L15.jpg,1100_01_L15_01.json,0.0,0
43399,1100,F,28,L30,2136,3216,1,327.0,843.0,1261.0,1193.0,1,이마,1100_01_L30.jpg,1100_01_L30_01.json,0.0,0
43408,1100,F,28,R15,2136,3216,1,671.0,833.0,1764.0,1209.0,1,이마,1100_01_R15.jpg,1100_01_R15_01.json,0.0,0


In [3]:
# original label
print(df_forehead['forehead_wrinkle'].unique())

#label rebalancing
print(df_forehead['adj_forehead_wrinkle_2'].unique()) ### 그래프로 하면 더 좋았겠음

class_counts = df_forehead['adj_forehead_wrinkle_2'].value_counts(sort=False).to_dict()
num_samples = sum(class_counts.values())
class_weights = {l: round(num_samples/class_counts[l], 2) for l in class_counts.keys()}

#print(f'cls_cnts: {len(class_counts)}\n num_samples:{num_samples}')

labels =  df_forehead['adj_forehead_wrinkle_2'].to_list()
weights = [class_weights[labels[i]] for i in range(int(num_samples))] 
sampler = torch.utils.data.WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))

print('class weight for each label :' , class_weights)

<IntegerArray>
[3, 1, 0, 2, 4, 5, 6]
Length: 7, dtype: Int64
<IntegerArray>
[1, 0, 2, 3]
Length: 4, dtype: Int64
class weight for each label : {1: 2.46, 0: 2.46, 2: 12.7, 3: 9.28}


In [7]:
import cv2
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, df, transforms=None):
        """
        Args:
        - df: DataFrame containing image paths, labels, and bounding box coordinates.
        - transforms: Any transformations to be applied to the images.
        """
        self.df = df
        self.df['path'] = '/Users/jaeeunlee/Documents/ds_study/ds_study/zerobase 프로젝트/deeplearning project/4. skin_digital/dataset/img/'+ df_forehead['image_name']
        self.transforms = transforms
        
    def __getitem__(self, index):
        # Fetch the row corresponding to this index
        row = self.df.iloc[index]
        
        # Extract image path and bounding box coordinates from the row
        img_path = row['path']
        bbox_x_min = int(row['bbox_x_min'])
        bbox_y_min = int(row['bbox_y_min'])
        bbox_x_max = int(row['bbox_x_max'])
        bbox_y_max = int(row['bbox_y_max'])

        # Read and crop the image using cv2
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        
        # Crop the image using the bounding box coordinates
        cropped_image = image[bbox_y_min:bbox_y_max, bbox_x_min:bbox_x_max]
        
        # Apply any transformations if they are provided
        if self.transforms is not None:
            cropped_image = self.transforms(cropped_image)  # 이미지 자체를 전달
        
        # If the label is available, return the cropped image and the label
        if 'forehead_wrinkle' in row:
            label = row['adj_forehead_wrinkle_2']
            return cropped_image, label
        else:
            return cropped_image
        
    def __len__(self):
        return len(self.df)

In [8]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split

# Example transformations (you can modify as needed)
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(0.5, 0.5)
])

# Create the dataset
dataset = CustomDataset(df=df_forehead, transforms=data_transforms)  # --> transform 나중에 해도 되는거 아닌가..? 그래야 보여줄 수 있을 거 같은뎅.....

train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = int(0.2 * len(dataset))

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Example of how to get an image and label
cropped_image, label = dataset[0]

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(len(train_loader)) #4825 * 0.8 / 32 = 121
print(len(val_loader)) #4825 * 0.2 / 32 = 31

#첫 한세트 테스트 하기 
for images, labels in train_loader :
    break

print(images.shape) #1계 텐서 변환 전(변환 ㅅ)
print(labels.shape)

91
31
torch.Size([32, 3, 224, 224])
torch.Size([32])


In [10]:
def evaluate_testset(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    test_acc = 0
    total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            predicted = torch.max(outputs, 1)[1]

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            test_loss += loss.item()
            test_acc += (predicted == labels).sum().item()
            total += len(labels)

    avg_test_loss = test_loss / total
    avg_test_acc = test_acc / total

    print(f'Test loss: {avg_test_loss:.5f}, Test accuracy: {avg_test_acc:.5f}')
    return avg_test_loss, avg_test_acc, all_preds, all_labels


#### 뭔가 이상하게 잘 안풀리는 중인데...

In [11]:
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = nn.Linear(model.last_channel, 4)  # 클래스 수 4로 설정
model.load_state_dict(torch.load("mobilnet_v2_trained_model.pth", map_location=device))
model = model.to(device)

weights = torch.tensor([2.46, 2.46, 12.7, 9.28]) #이마 class_weight
criterion = nn.CrossEntropyLoss(weight=weights, reduction='mean')
model.eval()

RuntimeError: Error(s) in loading state_dict for MobileNetV2:
	Unexpected key(s) in state_dict: "classifier.4.weight", "classifier.4.bias", "classifier.5.weight", "classifier.5.bias", "classifier.5.running_mean", "classifier.5.running_var", "classifier.5.num_batches_tracked", "classifier.8.weight", "classifier.8.bias", "classifier.0.weight", "classifier.0.bias", "classifier.1.running_mean", "classifier.1.running_var", "classifier.1.num_batches_tracked". 
	size mismatch for classifier.1.weight: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([4, 1280]).
	size mismatch for classifier.1.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([4]).